# CODE LLAMA 7B FINE TUNING - ALPACA 20K DATASET

## IMPORTS

In [ ]:
!pip install -q -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.34 trl==0.7.2 wandb
!pip install huggingface_hub
!pip install scipy

In [1]:
import os
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = "code_llama_alpaca_20K"
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find code_llama_alpaca_20K.
wandb: Currently logged in as: arjuntheprogrammer. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    CodeLlamaTokenizer,
    default_data_collator,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    BitsAndBytesConfig,
    AutoTokenizer,
)
from contextlib import nullcontext
from tqdm import tqdm
import json
import copy
import datasets
from peft import LoraConfig, PeftConfig
from transformers import default_data_collator, Trainer

/opt/conda/envs/code_llama_alpaca_20K/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/code_llama_alpaca_20K/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## PRETRAINED MODEL AND DATASET SETUP

In [4]:
model_name = "codellama/CodeLlama-7b-hf"
dataset_id = "HuggingFaceH4/CodeAlpaca_20K"

# compute_dtype = getattr(torch, "float16")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtyp=torch.bfloat16
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True
#     quantization_config=bnb_config,
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.90s/it]


In [10]:
print(tokenizer)

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


CodeLlamaTokenizerFast(name_or_path='codellama/CodeLlama-7b-hf', vocab_size=32016, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['▁<PRE>', '▁<MID>', '▁<SUF>', '▁<EOT>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32007: AddedToken("▁<PRE>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	32008: AddedToken("▁<SUF>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
	32009: AddedToken("▁<MID>", rstrip=True, lstrip=True

## PRE TRAINED MODEL INFERENCE

In [14]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    tokenizer=tokenizer,
)

sequences = pipeline(
    'import socket\n\ndef ping_exponential_backoff(host: str):',
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500,

)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

Result: import socket

def ping_exponential_backoff(host: str):
    """
    Ping a host with exponential backoff.

    :param host: host to ping
    :return: True if host is reachable, False otherwise
    """
    for i in range(1, 10):
        try:
            socket.create_connection((host, 80), 2).close()
            return True
        except OSError as e:
            if i < 10:
                time.sleep(2 ** i)
            else:
                raise e


def get_host_ip(host: str):
    """
    Get the IP address of a host.

    :param host: host to get the IP address of
    :return: IP address of host
    """
    return socket.gethostbyname(host)


def get_host_name(host: str):
    """
    Get the hostname of a host.

    :param host: host to get the hostname of
    :return: hostname of host
    """
    return socket.gethostname(host)


def get_host_fqdn(host: str):
    """
    Get the fully qualified domain name of a host.

    :param host: host to get the fully qualified domain 

## SETUP DATASET

In [5]:
def get_preprocessed_cmg_history(dataset_id, tokenizer, split):
    dataset = datasets.load_dataset(dataset_id, split=split)

    def apply_prompt_template(sample):
        return {
            "prompt": sample["prompt"],
            "message": sample["completion"],
        }

    dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))

    # mx = 0

    def tokenize_add_label(sample):
        prompt = tokenizer.encode(tokenizer.bos_token + sample["prompt"], add_special_tokens=False, max_length=200, truncation=True)
        message = tokenizer.encode(sample["message"] +  tokenizer.eos_token, max_length=400, truncation=True, add_special_tokens=False)
        max_length = 601 - len(prompt) - len(message)
        # mx = max(mx, len(prompt) + len(message))
        if max_length < 0:
            print("OK")

        pad = tokenizer.encode(tokenizer.eos_token, add_special_tokens=False, max_length=max_length, padding='max_length', truncation=True)

        sample = {
            "input_ids": prompt + message + pad,
            "attention_mask" : [1] * (len(prompt) + len(message) + len(pad)),
            "labels": [-100] * len(prompt) + message + [-100] * len(pad),
            }

        return sample

    dataset = dataset.map(tokenize_add_label, remove_columns=list(dataset.features))

    # print(mx)
    return dataset

In [6]:
train_dataset = get_preprocessed_cmg_history(dataset_id, tokenizer, 'train')


Map: 100%|██████████| 18019/18019 [00:15<00:00, 1177.36 examples/s]


## SETUP TRAINING CONFIG

In [7]:
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=4,
        lora_alpha=64,
        lora_dropout=0.1,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config


model, lora_config = create_peft_config(model)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=0.5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    group_by_length=True,
    logging_strategy="steps",
    save_strategy="no",
    gradient_checkpointing=False,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    data_collator=default_data_collator,
)

/opt/conda/envs/code_llama_alpaca_20K/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 2,097,152 || all params: 6,740,643,840 || trainable%: 0.031112042851977773


In [8]:
import torch;
print(torch.__version__)


2.3.0+cu121


## START TRAINING

Training Time: 1 hour 47 Mins

GPU Memeory USED: 11.6GB

In [15]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

/opt/conda/envs/code_llama_alpaca_20K/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/envs/code_llama_alpaca_20K/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,0.880400
20,0.615900
30,0.559500
40,0.588500
50,0.528700
60,0.455300
70,0.504900
80,0.435700
90,0.472500
100,0.432100


## FINETUNED MODEL INFERENCE

In [16]:
import time

# Before executing the code
start_time = time.time()
print("start_time:", start_time)


## evaluate the model
model.eval()
eval_prompt = """Create a Python class with the following attributes: firstname, lastname and address.
"""
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    output = tokenizer.decode(model.generate(**model_input, max_new_tokens=400, pad_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)
print("output:", output)


# After executing the code
end_time = time.time()
print("end_time:", end_time)

# Calculate the execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

start_time: 1717157111.0090144


/opt/conda/envs/code_llama_alpaca_20K/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


output: Create a Python class with the following attributes: firstname, lastname and address.
 class Person:
    def __init__(self, firstname, lastname, address):
        self.firstname = firstname
        self.lastname = lastname
        self.address = address
end_time: 1717157122.3060226
Execution time: 11.30 seconds


## SAVE MODEL TO HUGGING FACE

In [17]:
# !hugging-face-cli login --token xxx
from huggingface_hub import login
login(token="***")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/cto_auraml_com/.cache/huggingface/token
Login successful


In [19]:
model.push_to_hub('trained-model', use_temp_dir=False)
tokenizer.push_to_hub('trained-model', use_temp_dir=False)

adapter_model.bin: 100%|██████████| 8.43M/8.43M [00:02<00:00, 3.56MB/s]


CommitInfo(commit_url='https://huggingface.co/arjuntheprogrammer/trained-model/commit/681585b24fcff61087f57596c3bf05c18cbc2dc3', commit_message='Upload tokenizer', commit_description='', oid='681585b24fcff61087f57596c3bf05c18cbc2dc3', pr_url=None, pr_revision=None, pr_num=None)